#### Importando libraries, configurando webscraper and abrindo navegador

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import time

#setting driver
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
options.add_argument('--disable-gpu')

#creating driver
s = Service('/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.maximize_window()
driver.get("https://www.vagas.com.br/vagas-de-?ordenar_por=mais_recentes")

time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web

#### Clicando em "See more posts" ("Ver mais vagas")

In [ ]:
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec

j = 0

while j <= 15:
    timeout = 30
    # expand_button_xpath = '/html/body/div[1]/div/main/section/button'
    element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.ID, 'maisVagas')))
    driver.execute_script("arguments[0].click();", element)
    
    j += 1
    time.sleep(2)

screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break

#### Compilando links

In [ ]:
wait = WebDriverWait(driver, 5)

links=[]
new_links = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'link-detalhes-vaga')))
links.extend([l.get_attribute("href") for l in new_links])
print(links)

#### Criando dataframe e inserindo dados

In [ ]:
from tqdm.notebook import tqdm_notebook
import time

df_job = pd.DataFrame() # Creating empty dataframe

for i, l in enumerate(tqdm_notebook(links)):
    driver.get(l)
    time.sleep(5) 
    #JOB DESCRIPTION
    try:
        job_desc = driver.find_elements(By.CLASS_NAME, 'job-description__text')
        for ele in job_desc:
            desc = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        desc = ''
    #JOB TITLE
    try: 
        job_title = driver.find_elements(By.CLASS_NAME, 'job-shortdescription__title')
        for ele in job_title:
            title = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        title = ''
    #COMPANY
    try: 
        job_empresa = driver.find_elements(By.CLASS_NAME, 'job-shortdescription__company')
        for ele in job_empresa:
            empresa = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        empresa = ''
    #LOCATION
    try: 
        job_localidade = driver.find_elements(By.CLASS_NAME, 'info-localizacao')
        for ele in job_localidade:
            localidade = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        localidade = ''
    #POST DATE
    try: 
        job_data_publ = driver.find_elements(By.CLASS_NAME, 'job-breadcrumb__item--published')
        for ele in job_data_publ:
            data_publ = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        data_publ = ''
    #SENIORITY LEVEL
    try: 
        job_experiencia = driver.find_elements(By.CLASS_NAME, 'job-hierarchylist__item--level')
        for ele in job_experiencia:
            experiencia = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        experiencia = ''

    ## creating dictionnary with job data
    dict = {'Plataforma': 'VagasCom','Titulo': title.text, 'Empresa': empresa.text, 'Localidade': localidade.text, 'Data_Publicacao': data_publ.text.replace('Publicada ', '').replace('Anunciada: há ', '').replace('Anunciada: ', '').capitalize(), 'Descricao': desc.text, 'Nivel_experiencia': experiencia.text, 'Tipo_emprego': '', 'Funcao': '', 'Setor': ''}
    ## inserting dictionnary in dataframe
    df_job = df_job.append(dict, ignore_index = True)

#### Salvando arquivo com CSV formato

In [ ]:
from datetime import datetime
currentDateTime = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
df_job.to_csv(f"./Data/Extracao_VagasCom_{currentDateTime}.csv", index = False, encoding='utf-8-sig', sep=';')

driver.quit()

In [ ]:
df_job